## Weather Data : Initial Visualization

### For Southern Plains States: Texas, Oklahoma, Kansas

In [4]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

from pyspark import SparkContext
from pyspark.sql import *
import pyspark.sql
sqlContext = SQLContext(sc)

import numpy as np
from lib.numpy_pack import packArray,unpackArray
from lib.spark_PCA import computeCov
from lib.computeStatistics import *

Populating the interactive namespace from numpy and matplotlib


In [5]:
import warnings  # Suppress Warnings
warnings.filterwarnings('ignore')

_figsize=(10,7)

## Read Data

### Read Raw data for state - starting with Texas

In [20]:
state='TX'
data_dir='../Data/Weather'

tarname=state+'.tgz'
parquet=state+'.parquet'

#%mkdir -p $data_dir
#!rm -rf $data_dir/$tarname

#command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s"%(tarname,data_dir,tarname)
#print(command)
#!$command
#!ls -lh $data_dir/$tarname

In [21]:
print(parquet)
weather_df=sqlContext.read.parquet(data_dir+'/'+parquet)
#weather_df=weather_df.drop('State') # we drop State because it already exists in "Stations".

TX.parquet


### read statistics information for state.

In [ ]:
#weather_df=weather_df.drop('latitude').drop('longitude').drop('dist_coast').drop('state')
#weather_df.show(3)

In [22]:
sqlContext.registerDataFrameAsTable(weather_df,'jdf')

# get all SNOW (snowfall) measurements for all stations in Texas
measure = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Measurement='%s' 
"""%(measure )

#sqlContext.sql(Query).show(5)


In [ ]:
#sqlContext.sql(Query).count()

In [23]:
# get all measurements for years 1970-2010
year_low = 1970
year_high = 2010
measurement = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Year >='%d' 
and Year <='%d'
and Measurement='%s' 
"""%(year_low, year_high, measurement )

sqlContext.sql(Query).show(5)

+-----------+-----------+----+--------------------+-----------------+----------------+-------------------+-----------------+-----+----------+
|    Station|Measurement|Year|              Values|       dist_coast|        latitude|          longitude|        elevation|state|      name|
+-----------+-----------+----+--------------------+-----------------+----------------+-------------------+-----------------+-----+----------+
|USC00411416|       SNOW|1970|[00 00 00 00 00 0...|602.5280151367188|30.1382999420166|-104.68219757080078|876.9000244140625|   TX|CANDELARIA|
|USC00411416|       SNOW|1971|[00 00 00 00 00 0...|602.5280151367188|30.1382999420166|-104.68219757080078|876.9000244140625|   TX|CANDELARIA|
|USC00411416|       SNOW|1972|[00 00 00 00 00 0...|602.5280151367188|30.1382999420166|-104.68219757080078|876.9000244140625|   TX|CANDELARIA|
|USC00411416|       SNOW|1973|[00 00 00 7E 00 7...|602.5280151367188|30.1382999420166|-104.68219757080078|876.9000244140625|   TX|CANDELARIA|
|USC00

In [8]:
#sqlContext.sql(Query).count()

17528

## Convert to a Pandas dataframe. 

In [24]:
pandas_df=sqlContext.sql(Query).toPandas()
pandas_df=pandas_df.set_index('Measurement')
#pandas_df

In [9]:
pandas_df['name'].value_counts()

CORPUS CHRISTI              46
NACOGDOCHES                 42
TOWN BLUFF DAM              42
BALMORHEA                   41
ROTAN                       41
ELGIN                       41
CLARKSVILLE 2NE             41
LIVINGSTON 2 NNE            41
FALCON DAM                  41
MADISONVILLE                41
ALPINE                      41
WACO DAM                    41
CROCKETT                    41
NAVARRO MILLS DAM           41
MARSHALL                    41
KINGSVILLE NAAS             41
STRATFORD                   41
ALBANY                      41
DANEVANG 1 W                41
LANGTRY                     41
RAYMONDVILLE                41
BAY CITY WTR WKS            41
PERRYTON                    41
FOWLERTON                   41
LULING                      41
DAINGERFIELD 9 S            41
CROSBYTON                   41
POTEET                      41
ROBERT LEE                  41
BALLINGER 2 NW              41
                            ..
LAZY H RCH                   2
PRESIDIO

In [25]:
# Only keep stations which have values for each year in 1970-2010. 
pandas_df = pandas_df[pandas_df['name'].isin(pandas_df['name'].value_counts()[pandas_df['name'].value_counts()==41].index)]
#pandas_df

In [26]:
pandas_df['TX_SNOW_Means'] = list(pd.DataFrame(pandas_df['Values'].values.tolist()).mean(1))
#pandas_df['SNOW_Means']

In [12]:
pandas_df

,Station,Year,Values,dist_coast,latitude,longitude,elevation,state,name,TX_SNOW_Means
Measurement,,,,,,,,,,
SNOW,USC00411416,1970,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1971,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1972,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1973,"[0, 0, 0, 126, 0, 126, 0, 0, 0, 0, 0, 0, 0, 0,...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.517808
SNOW,USC00411416,1974,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1976,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1977,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000
SNOW,USC00411416,1978,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",602.528015,30.1383,-104.682198,876.900024,TX,CANDELARIA,0.000000


In [27]:
year_means = pd.DataFrame(pandas_df.groupby('Year')['TX_SNOW_Means'].mean())
year_means

,TX_SNOW_Means
Year,
1970,0.684524
1971,1.279367
1972,1.367519
1973,4.240347
1974,1.328049
1975,2.064804
1976,1.204000
1977,1.786174
1978,3.672323


# Get data for Oklahoma: 

In [6]:
state='OK'
data_dir='../Data/Weather'

tarname=state+'.tgz'
parquet=state+'.parquet'

#%mkdir -p $data_dir
#!rm -rf $data_dir/$tarname

#command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s"%(tarname,data_dir,tarname)
#print(command)
#!$command
#!ls -lh $data_dir/$tarname

In [7]:
!ls $data_dir

decon_NY_SNWD.parquet	    south_plains_snow_data_large.csv
KS.parquet		    south_plains_snow_data.parquet
KS.tgz			    stations.parquet
ND.parquet		    STAT_NY.pickle
ND.tgz			    STAT_TX.pickle
NY.parquet		    TX.parquet
NY.tgz			    TX.tgz
OK.parquet		    US_stations.tsv
OK.tgz			    Weather_Stations.parquet
recon_NY_SNWD.parquet	    Weather_stations.tgz
south_plains_snow_data	    Weather_Stations.tgz
south_plains_snow_data.csv


In [8]:
OK_weather_df=sqlContext.read.parquet(data_dir+'/'+parquet)

In [9]:
#print('number of rows=',OK_weather_df.cache().count())
#OK_weather_df.show(5)
#OK_weather_df=OK_weather_df.drop('latitude').drop('longitude').drop('dist_coast').drop('state')
#OK_weather_df.show(3)

In [10]:
sqlContext.registerDataFrameAsTable(OK_weather_df,'jdf')

In [11]:
# get all SNOW (snowfall) measurements for all stations in Texas
measure = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Measurement='%s' 
"""%(measure )

sqlContext.sql(Query).show(5)

+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|       name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------+
|USC00348951|       SNOW|1942|[00 7E 00 00 00 0...|631.1580200195312|35.13330078125|-97.05000305175781|313.8999938964844|   OK|TRIBBEY 1 N|
|USC00348951|       SNOW|1943|[00 7E 00 7E 00 7...|631.1580200195312|35.13330078125|-97.05000305175781|313.8999938964844|   OK|TRIBBEY 1 N|
|USC00348951|       SNOW|1944|[00 7E 00 7E 00 7...|631.1580200195312|35.13330078125|-97.05000305175781|313.8999938964844|   OK|TRIBBEY 1 N|
|USC00348951|       SNOW|1945|[00 7E 00 7E 00 7...|631.1580200195312|35.13330078125|-97.05000305175781|313.8999938964844|   OK|TRIBBEY 1 N|
|USC00348951|       

In [12]:
# get all measurements for years 1970-2010
year_low = 1970
year_high = 2010
measurement = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Year >='%d' 
and Year <='%d'
and Measurement='%s' 
"""%(year_low, year_high, measurement )

sqlContext.sql(Query).show(5)

+-----------+-----------+----+--------------------+-----------------+----------------+------------------+------------------+-----+------------+
|    Station|Measurement|Year|              Values|       dist_coast|        latitude|         longitude|         elevation|state|        name|
+-----------+-----------+----+--------------------+-----------------+----------------+------------------+------------------+-----+------------+
|USC00345563|       SNOW|1970|[00 7E 00 7E 00 7...|506.7170104980469|33.8760986328125|-97.16419982910156|244.39999389648438|   OK|MARIETTA 5SW|
|USC00345563|       SNOW|1971|[00 00 00 00 00 0...|506.7170104980469|33.8760986328125|-97.16419982910156|244.39999389648438|   OK|MARIETTA 5SW|
|USC00345563|       SNOW|1972|[00 7E 00 7E 00 7...|506.7170104980469|33.8760986328125|-97.16419982910156|244.39999389648438|   OK|MARIETTA 5SW|
|USC00345563|       SNOW|1973|[00 7E 00 7E 00 7...|506.7170104980469|33.8760986328125|-97.16419982910156|244.39999389648438|   OK|MARIET

In [13]:
#Convert to pandas dataframe 

OK_pandas_df=sqlContext.sql(Query).toPandas()
OK_pandas_df=OK_pandas_df.set_index('Measurement')
OK_pandas_df

# Only keep stations which have values for each year in 1970-2010. 
OK_pandas_df = OK_pandas_df[OK_pandas_df['name'].isin(OK_pandas_df['name'].value_counts()[OK_pandas_df['name'].value_counts()==41].index)]
OK_pandas_df


OK_pandas_df['OK_SNOW_Means'] = list(pd.DataFrame(OK_pandas_df['Values'].values.tolist()).mean(1))
#pandas_df['SNOW_Means']

OK_year_means = pd.DataFrame(OK_pandas_df.groupby('Year')['OK_SNOW_Means'].mean())
OK_year_means

,OK_SNOW_Means
Year,
1970,2.957012
1971,2.491294
1972,5.918331
1973,9.603262
1974,4.750613
1975,5.406363
1976,1.672909
1977,1.981182
1978,5.714366


# Get data for Kansas:

In [14]:
state='KS'
data_dir='../Data/Weather'

tarname=state+'.tgz'
parquet=state+'.parquet'

#%mkdir -p $data_dir
#!rm -rf $data_dir/$tarname

#command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s"%(tarname,data_dir,tarname)
#print(command)
#!$command
#!ls -lh $data_dir/$tarname

In [ ]:
!ls $data_dir

In [ ]:
cur_dir,=!pwd
%cd $data_dir
!tar -xzf $tarname
!du ./$parquet
%cd $cur_dir

In [15]:
KS_weather_df=sqlContext.read.parquet(data_dir+'/'+parquet)

In [ ]:
#print('number of rows=',KS_weather_df.cache().count())
#KS_weather_df=KS_weather_df.drop('latitude').drop('longitude').drop('dist_coast').drop('state')
#KS_weather_df.show(3)

In [16]:
sqlContext.registerDataFrameAsTable(KS_weather_df,'jdf')

In [17]:
# get all SNOW (snowfall) measurements for all stations in Texas
measure = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Measurement='%s' 
"""%(measure )

#sqlContext.sql(Query).show(5)

In [18]:
# get all measurements for years 1970-2010
year_low = 1970
year_high = 2010
measurement = 'SNOW'
Query="""
SELECT *
FROM jdf
WHERE Year >='%d' 
and Year <='%d'
and Measurement='%s' 
"""%(year_low, year_high, measurement )

#sqlContext.sql(Query).show(5)

In [19]:
#Convert to pandas dataframe 

KS_pandas_df=sqlContext.sql(Query).toPandas()
KS_pandas_df=KS_pandas_df.set_index('Measurement')
KS_pandas_df

# Only keep stations which have values for each year in 1970-2010. 
KS_pandas_df = KS_pandas_df[KS_pandas_df['name'].isin(KS_pandas_df['name'].value_counts()[KS_pandas_df['name'].value_counts()==41].index)]
KS_pandas_df


KS_pandas_df['KS_SNOW_Means'] = list(pd.DataFrame(KS_pandas_df['Values'].values.tolist()).mean(1))
#pandas_df['SNOW_Means']

KS_year_means = pd.DataFrame(KS_pandas_df.groupby('Year')['KS_SNOW_Means'].mean())
KS_year_means

,KS_SNOW_Means
Year,
1970,3.281713
1971,3.489650
1972,4.299261
1973,5.791292
1974,3.809285
1975,5.163459
1976,1.905229
1977,2.536780
1978,5.170026


In [ ]:
south_plains_df = pd.concat([year_means, OK_year_means, KS_year_means], axis=1)

In [ ]:
south_plains_df

In [ ]:
#south_plains_df.to_parquet(data_dir+'/'+"south_plains_snow_data.parquet")

In [ ]:
!ls $data_dir

In [ ]:
#south_plains_df.to_csv(data_dir+'/'+"south_plains_snow_data.csv")

In [ ]:
!ls $data_dir

In [28]:
south_plains_big_df = pd.concat([pandas_df, KS_pandas_df, OK_pandas_df])

In [29]:
south_plains_big_df

,KS_SNOW_Means,OK_SNOW_Means,Station,TX_SNOW_Means,Values,Year,dist_coast,elevation,latitude,longitude,name,state
Measurement,,,,,,,,,,,,
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1970,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1971,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1972,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.517808,"[0, 0, 0, 126, 0, 126, 0, 0, 0, 0, 0, 0, 0, 0,...",1973,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1974,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1975,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1976,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1977,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX
SNOW,NaN,NaN,USC00411416,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1978,602.528015,876.900024,30.138300,-104.682198,CANDELARIA,TX


In [30]:
south_plains_big_df.to_csv(data_dir+'/'+"south_plains_snow_data_large.csv")

In [31]:
#south_plains_big_df.to_parquet(data_dir+'/'+"south_plains_snow_data.parquet")

ArrowInvalid: Error inferring Arrow type for Python object array. Got Python object of type bytearray but can only handle these types: string, bool, float, int, date, time, decimal, list, array